# Import Packages

In [3]:
import pandas as pd
import numpy as np
import warnings 
import json
import yaml
warnings.simplefilter('ignore')

# Load Data

In [3]:
%%time
# The data is too big, we load 600000 rows every time
# train data
df = pd.read_csv('train_v2.csv', nrows = 600000)

CPU times: user 1min 8s, sys: 24.7 s, total: 1min 32s
Wall time: 1min 58s


In [4]:
df2 = pd.read_csv('train_v2.csv', nrows = 600000, skiprows = range(1, 600001))

In [5]:
df3 = pd.read_csv('train_v2.csv', nrows = 600000, skiprows = range(1, 600001+600000))

In [6]:
train = pd.concat([df, df2, df3], ignore_index=True)

In [8]:
# test data
test = pd.read_csv('test_v2.csv')

In [9]:
%%time
# combine train and test
combi = pd.concat([train, test], ignore_index=True)

CPU times: user 8.39 s, sys: 1min 19s, total: 1min 27s
Wall time: 3min 29s


In [52]:
combi.shape

(2109926, 13)

# Data Preprocessing

### Json data

customDimensions, device, geoNetwork, hits, totals, trafficSource

(1) customDimensions

In [10]:
def str_list_json(x):
    str_json = ''.join(list(x)[1:-1])
    json_ = yaml.load(str_json) # yaml is for nonstandarded json format
    return json_

In [11]:
%%time
jsons = combi['customDimensions'].apply(str_list_json)

CPU times: user 8min 42s, sys: 2.19 s, total: 8min 44s
Wall time: 8min 47s


In [12]:
%%time
jsons_list = [x if x != None else {'index': np.nan, 'value': np.nan} for x in jsons]

CPU times: user 403 ms, sys: 137 ms, total: 540 ms
Wall time: 561 ms


In [13]:
%%time
customDim = pd.DataFrame(jsons_list)
customDim.columns = ['cD_index', 'cD_value']

CPU times: user 2.08 s, sys: 721 ms, total: 2.8 s
Wall time: 2.97 s


(2) device, geoNetwork, totals, trafficSource

In [15]:
def str_json(x):
    return(json.loads(x))

In [16]:
jsons_device = combi['device'].apply(str_json)
device = pd.DataFrame(jsons_device.tolist())

In [19]:
device_colnames = []
for col in device.columns:
    device_colnames += ['d_' + col]
device.columns = device_colnames

(3) geoNetwork

In [21]:
%%time
jsons_geoN = combi['geoNetwork'].apply(str_json)
geoNetwork = pd.DataFrame(jsons_geoN.tolist())

CPU times: user 22.5 s, sys: 8.09 s, total: 30.6 s
Wall time: 33.2 s


In [22]:
geoNetwork_colnames = []
for col in geoNetwork.columns:
    geoNetwork_colnames += ['geo_' + col]
geoNetwork.columns = geoNetwork_colnames

(4) totals

In [25]:
%%time
jsons_totals = combi['totals'].apply(str_json)
totals = pd.DataFrame(jsons_totals.tolist())

CPU times: user 12.8 s, sys: 2.23 s, total: 15 s
Wall time: 16.1 s


In [30]:
totals_colnames = []
for col in totals.columns:
    totals_colnames += ['t_' + col]
totals.columns = totals_colnames

(5) trafficsource

In [33]:
%%time
jsons_traffic = combi['trafficSource'].apply(str_json)
trafficsource = pd.DataFrame(jsons_traffic.tolist())

CPU times: user 21.6 s, sys: 3.49 s, total: 25.1 s
Wall time: 27.7 s


In [34]:
traffic_colnames = []
for col in trafficsource.columns:
    traffic_colnames += ['ts_' + col]
trafficsource.columns = traffic_colnames

# New Dataframe by combining extracted features above

In [38]:
json_col = ['customDimensions', 'hits', 'device', 'geoNetwork', 'totals', 'trafficSource']
combi_new = combi.drop(json_col, axis=1)

In [39]:
combi_new = pd.concat([combi_new, customDim, device, geoNetwork, trafficsource, totals], axis = 1)

# Save data

In [51]:
combi_new.to_csv('combi.csv', index = False, header = True)

In [63]:
# train
train = combi_new[:len(train)]

# test
test = combi_new[len(train):]
test.reset_index(drop=True, inplace=True)

In [64]:
train.to_csv('train_nohits.csv', index=False, header=True)
test.to_csv('test_nohits.csv', index=False, header=True)